In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
#default_exp classify_image

In [6]:
#export
from food.tools import *
from food.paths import *

import pandas as pd
import numpy as np
from food.psql import *

# !nbdev_build_lib
import requests
from food.clipmodel import image2clip
from PIL import Image

In [7]:
#export
from food.milvus import collection
collection.load()

In [8]:
#export
def search_by_clip(clip, topk=10, query_expr='',search_params = {'ef':10000}): #32768
    search_params = {"metric_type": "IP",
                     "params": search_params}

    print(query_expr)
    results = collection.search(clip[None,], "clip", search_params, limit=topk, expr=query_expr, output_fields=["id"])
    results = results[0]
    return list(results.distances), results.ids

def get_metadata(ids):
    q = f"""select * from foods_big where id in {tuple(ids)}"""
    return pd.read_sql(q,engine)

In [9]:
#export
def search_image(url=None,path=None):
    if url:
        response = requests.get(url, stream=True)
        image = Image.open(response.raw)
    elif path:
        image = Image.open(path)
    clip = image2clip(image)
    results = search_by_clip(clip.numpy())
    df = get_metadata(results[1])
    df['score'] = results[0]
    return df

In [10]:
#export
def get_calories(url=None,path=None):
    r = search_image(url=url)
    r = r[(r['energy_kcal_100g']>r['energy_kcal_100g'].quantile(0.3))&(r['energy_kcal_100g']<r['energy_kcal_100g'].quantile(0.9))]
    rr = r[['energy_kcal_100g','proteins_100g','fat_100g','carbohydrates_100g']].mean().round(2)
    rr['ids'] =  r['id'].tolist()
    rr['scores'] =  r['score'].tolist()
    rr['names']  =  r['product_name'].tolist()
    return rr

In [11]:
url = 'https://cdn.shopify.com/s/files/1/0943/7472/products/12540011_0_640x640_b3b5eedd-b502-4b22-8fb4-70c040c4798b_x700.jpg?v=1614184159'

In [12]:
search_image(url)

,id,product_name,keywords,ingredients_text,categories,food_groups,energy_kcal_100g,proteins_100g,fat_100g,carbohydrates_100g,clip,score
0,35872,"Lindt, excellence, black currant dark chocolat...","bar, confectionerie, snack, dark, sweet, curra...","Sugar, chocolate, black currant pieces (black ...","Snacks, Sweet snacks, Confectioneries, Chocola...",sweets,500.00,5.0,30.000,57.500,"[0.013169993, 0.039979216, 0.051202036, 0.0358...",0.379934
1,227195,Lindt Excellence Dark Chocolate Bar - 70%,"ariel, snack, it, excellence, fine, vegan, bar...","cocoa mass, sugar, cocoa butter, natural bourb...","Snacks, Sweet snacks, Cocoa and its products, ...",chocolate-products,566.00,9.5,41.000,34.000,"[-0.0012109026, 0.03580227, 0.032108422, 0.031...",0.364851
2,227314,Lindt excellence 90% cacao,"noir, tablette, vert, derive, point, cacao, su...","pâte de cacao, beurre de cacao, cacao maigre, ...","Snacks,Snacks sucrés,Cacao et dérivés,Chocolat...",chocolate-products,592.00,10.0,55.000,14.000,"[0.0117876865, 0.039992243, 0.040229462, -0.01...",0.360589
3,546920,Lindt Excellence Mild 85% Probierpreis -23%,"85, dark-chocolate, 23, probierprei, mild, exc...","Kakao: 85% mindestens Zutaten: Kakaomasse, Kak...","Snacks, Snacks sucrés, Cacao et dérivés, Choco...",chocolate-products,603.00,8.9,55.000,19.000,"[0.00069913967, 0.027529106, 0.02723215, 0.004...",0.359219
4,670513,Lindt EXCELLENCE 95% cocoa,"95, snack, excellence, lindt, dark, sweet, cho...","cocoa mass, cocoa butter, cocoa powder, sugar,...","Snacks, Sweet snacks, Cocoa and its products, ...",chocolate-products,620.00,12.5,56.875,9.375,"[0.010039008, 0.040425982, 0.051311944, 0.0554...",0.352891
5,865481,Lindt 95% cacao bold dark chocolate,"lindt, 95, chocolate, cacao, dark, bold",NaN,NaN,NaN,625.00,12.5,56.250,25.000,"[0.01681229, 0.015542973, 0.042658202, -0.0022...",0.349986
6,2910991,Lindt excellence 70% cacao noir intense,"noir, chocolate, 70, product, sweet, excellenc...",,"Snacks,Sweet snacks,Cocoa and its products,Cho...",chocolate-products,566.00,9.5,41.000,34.000,"[0.012975434, 0.03518995, 0.027368044, 0.01126...",0.348653
7,3016997,Lindt excellence mint intense,"excellence, intense, mint, dark-chocolate, spr...","Sugar, cocoa mass, cocoa butter, anhydrous mil...",en:dark-chocolates,chocolate-products,537.00,5.7,32.000,52.000,"[-0.017321825, 0.05865552, 0.07431627, 0.06800...",0.347456
8,3087811,Шоколад Lindt Excellence 85% какао,"85, excellence, шоколад, lindt, какао",NaN,NaN,NaN,550.34,11.0,46.000,19.000,"[0.005217102, 0.055546798, 0.030605648, 0.0267...",0.346110
9,3177210,85% cacao RICH DARK,"lindt, 85, dark, excellence, rich, cacao",NaN,NaN,NaN,584.00,12.5,46.000,22.000,"[0.044333573, 0.010418254, 0.033181865, -0.006...",0.344328


In [13]:
get_calories(url)

energy_kcal_100g                                                  588.5
proteins_100g                                                     10.48
fat_100g                                                          49.15
carbohydrates_100g                                                22.06
ids                   [227195, 227314, 546920, 670513, 2910991, 3177...
scores                [0.3648509979248047, 0.36058855056762695, 0.35...
names                 [Lindt Excellence Dark Chocolate Bar - 70%, Li...
dtype: object

In [20]:
pd.read_sql("select count(*) from foods_big where energy_kcal_100g is null",engine)

,count
0,0


In [118]:
pd.read_sql("select * from foods_big where clip is not null",engine)

,id,product_name,keywords,ingredients_text,categories,food_groups,energy_kcal_100g,proteins_100g,fat_100g,carbohydrates_100g,clip


In [48]:
df = pd.read_sql("select * from foods_big",engine).set_index('id')

In [103]:
text = df[['product_name','keywords','ingredients_text','categories','food_groups']].fillna("")
for c in text.columns: text[c] = text[c].str.replace('NaN','')
text = text['product_name']+ '. ' + text['food_groups']+ '. ' + text['keywords'] + '. ' + text['ingredients_text'] + '. '+ text['categories']

In [117]:
text.loc[text.sample(1).index[0]]

"panna cotta fruits rouges. dairy-desserts. panna, de, au, produit, lacte, dessert, rouge, laitier, franprix, framboise, couli, la, fruit, cotta. Crème fraîche 31,7% - eau - sirop de glucose - framboises 10,4% - sucre - lait écrémé en poudre - huile de coco - alcool - amidon de tapioca - épaississants : carraghénanes, alginate de sodium - correcteur d’acidité : acide citrique - gélifiant : pectines - arômes naturels - émulsifiant : mono - et diglycérides d’acides gras - extrait de sureau - conservateur : sorbate de potassium.Teneur en alcool : 1,0%.Traces d'œuf et blé.Les informations en gras sont destinées aux personnes intolérantes ou allergiques.. Produits laitiers, Desserts, Desserts lactés, Panna cottas, Panna cottas au coulis de fruits, Panna cottas à la framboise"

In [14]:
!nbdev_build_lib

Converted 00_paths.ipynb.
Converted 00_progress_check.ipynb.
Converted 00_psql.ipynb.
Converted 00_tools.ipynb.
Converted 04_milvus.ipynb.
Converted 06_clip.ipynb.
Converted 0_template.ipynb.
Converted bot.ipynb.
Converted classfy_image.ipynb.
Converted inference.ipynb.
Converted milvus_update.ipynb.
Converted openfoodfacts_json_parsing.ipynb.
